In [1]:
%load_ext autoreload
%autoreload 2
import simplemir.fileutils as fu
import simplemir.music21utils as mu

from sklearn.cluster import KMeans
import numpy as np

import json
import pprint
import copy
import time
import random

from music21 import stream, instrument,note, pitch

# Load json analysis and score files

In [2]:
# Load all Json analysis objects into a single list
start  = time.time()
print('Starting')

path_dirs_list = ['wtca','wtcb','wtcd','wtce','wtcf']
data =[]
for p in path_dirs_list: 
    with open(p+'.json', 'r') as f:
        datastore = json.load(f)
        json_string = json.dumps(datastore)
        datastore = json.loads(json_string)
        data = data + copy.deepcopy( datastore)

# Create list of descriptors arrays
descrips = [d['descriptors']['master'] for d in data]


# Get files from all directories in one list
superPathList =[]
for p in path_dirs_list: 
    superPathList += fu.get_list_files(p, extension="xml")

# Get scores from path list
scoresListJSON = mu.get_scores_from_paths_json(superPathList)

# Get measures json list
allMeasures = mu.getMeasuresListJson(scoresListJSON)

print('Time:',int(time.time()-start),'sec') # Around 23 secs


Starting
Time: 29 sec


In [182]:
scoresListJSON[0]['score'].show('txt')

{0.0} <music21.metadata.Metadata object at 0x7f8633733eb8>
{0.0} <music21.stream.Part Piano>
    {0.0} <music21.instrument.Instrument P1: Piano: Piano>
    {0.0} <music21.stream.Measure 1 offset=0.0>
        {0.0} <music21.layout.SystemLayout>
        {0.0} <music21.clef.TrebleClef>
        {0.0} <music21.tempo.MetronomeMark larghetto Quarter=60.0>
        {0.0} <music21.key.KeySignature of 2 flats>
        {0.0} <music21.meter.TimeSignature 4/4>
        {0.0} <music21.note.Rest rest>
    {4.0} <music21.stream.Measure 1148 offset=4.0>
        {0.0} <music21.clef.TrebleClef>
        {0.0} <music21.tempo.MetronomeMark larghetto Quarter=60.0>
        {0.0} <music21.key.KeySignature of 2 flats>
        {0.0} <music21.meter.TimeSignature 4/4>
        {0.0} <music21.note.Rest rest>
        {2.0} <music21.note.Note G>
        {2.5} <music21.note.Note B->
        {3.0} <music21.note.Note D>
        {3.5} <music21.note.Note D->
    {8.0} <music21.stream.Measure 409 offset=8.0>
        {0.0} <mu

In [3]:
start  = time.time()
print('Starting')

# Begin clustering
X = np.array(descrips)
numClasses=20
kmeans = KMeans(n_clusters=numClasses, random_state=0).fit(X)
clusters =[]
for i,k in enumerate(kmeans.labels_):
    clusters.append({
        'index':i,
        'class':k
    })
clust =[]
for i in range(numClasses):
    clust.append(list(filter(lambda c: c['class'] == i, clusters)))

clustScores = []
for c in clust:
    clustScores.append( [data[int(e['index'])] for i,e in enumerate(c)])

# Verify that descriptors are correct
# masters = [ c['descriptors']['master'] for c in clustScores[15]]
# kmeans.predict(masters)

print('Time:',int(time.time()-start),'sec') # Around 3 sec


Starting
Time: 3 sec


In [91]:
clustScores[0][:10]

[{'path': 'wtca/wtc1f20.xml',
  'part': 'Piano',
  'number': 7,
  'measure': 'seeOtherList',
  'descriptors': {'numberNotes': 10,
   'key': [0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0],
   'pcCount': [1, 0, 1, 0, 1, 0, 1, 0, 2, 3, 0, 1],
   'melIntervs': [0, 4, 4, 0, 0, 0, 0, 1, 0, 0, 0, 0],
   'pitchSpan': [66, 76],
   'pitchRanges': [0, 10],
   'master': [10,
    0,
    0,
    0,
    0,
    20,
    0,
    0,
    0,
    0,
    0,
    0,
    0,
    1,
    0,
    1,
    0,
    1,
    0,
    1,
    0,
    2,
    3,
    0,
    1,
    0,
    4,
    4,
    0,
    0,
    0,
    0,
    1,
    0,
    0,
    0,
    0,
    66,
    76,
    0,
    10]}},
 {'path': 'wtca/wtc1f20.xml',
  'part': 'Piano',
  'number': 60,
  'measure': 'seeOtherList',
  'descriptors': {'numberNotes': 8,
   'key': [0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0],
   'pcCount': [1, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 4],
   'melIntervs': [0, 2, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0],
   'pitchSpan': [68, 72],
   'pitchRanges': [0, 4],
   'master': [8,
 

# Generate scores compositions

Here we will generate the score

- For each cluster we will hear random concatenations
- Filter each cluster by instrument (range)
- Generate parts
- Generate whole score

In [7]:
def sortNumNotes(meas): 
    return meas['descriptors']['numberNotes'] 
def sortRangeNotes(meas): 
    return meas['descriptors']['numberNotes'] 
def sortRangeNotes(meas): 
    return meas['descriptors']['numberNotes'] 
def averageNote(spanNotes):
    return (int(spanNotes[1])+int(spanNotes[0]))/2
def maxNote(m):
    maxNote = -1
    for e in m.recurse():
        if type(e)== note.Note:
            if e.pitch.midi > maxNote:
                maxNote =e.pitch.midi
    return maxNote
def minNote(m):
    minNote = 200
    for e in m.recurse():
        if type(e)== note.Note:
            if e.pitch.midi < minNote:
                minNote =e.pitch.midi
    return minNote

NUM_MEASURES = 50

for clusterNum in range(16,17):
    start  = time.time()
    print('Starting')
    print('Cluster ',clusterNum)
    random.shuffle(clustScores[clusterNum])

    partMeasures =[]
    for c in clustScores[clusterNum][:NUM_MEASURES]:
        partMeasures.append(mu.thisMeasure(c,allMeasures))
#     partMeasures.sort(key=sortNumNotes,reverse=False)


    print(len(partMeasures))
    streamRes = stream.Stream()
    partPiano = stream.Part()
    partPiano.insert(instrument.Piano())

    partViolin = stream.Part()
    partViolin.insert(instrument.Violin())

    partCello = stream.Part()
    partCello.insert(instrument.Violoncello())

    
    for i, m in enumerate(partMeasures):
        try:
            m['measure'].number = i+1
            aver =  averageNote(m['descriptors']['pitchSpan'])
            maxim = maxNote(m['measure'])
            minim = minNote(m['measure'])
#             print( minim, maxim)
            if aver in range(42,72):
                partPiano.append(m['measure'])
            if  minim > 59  and not mu.measureIsMultiVoice(m['measure']):
                partViolin.append(m['measure'])
            if maxim < 72 and not mu.measureIsMultiVoice(m['measure']):
                partCello.append(m['measure'])

        except:
#             print(e)
            True
#             print('Repetido pero no se aguite...')
    streamRes.append(partViolin)
    streamRes.append(partCello)
    streamRes.append(partPiano)
    streamRes = mu.remove_breaks(streamRes)
    # streamRes.write('xml')
    data=mu.stream_xml_string(streamRes)
    fu.save_file(data,'output/cluster-'+str(clusterNum)+'.xml')
    dataJSON = json.dumps(clustScores[clusterNum],)
    fu.save_file(dataJSON,'output/cluster-measures'+str(clusterNum)+'.json')

    
    print('Time:',int(time.time()-start),'sec') # Around 3 sec




Starting
Cluster  16
50
Time: 0 sec


In [154]:
# streamRes.write('xml')
data=mu.stream_xml_string(streamRes)
fu.save_file(data,'output/test.xml')

In [ ]:
streamRes.show()
